# Задание 4. Алгоритм Nearest Neighbors

В данном задании требуется самостоятельно реализовать алгоритм k-NN и применить его.

Импорт необходимых библиотек:

In [ ]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

Загрузим датасет:

In [ ]:
# Download dataset
data, labels = load_wine(return_X_y=True)

Разбейте ваши данные на тренировочную, валидационную и тестовую подвыборки.

Вам пригодится метод `train_test_split()`. Выделите на обучение $60\%$ данных, не забудьте про фиксирование `seed` генератора и *стратификацию* (параметры `random_state=42`, `stratify`).

In [ ]:
x_train, x_, y_train, y_ = train_test_split(
    data, labels, train_size=0.6, random_state=42, stratify=labels
)
# Your code here
x_val, x_test, y_val, y_test = train_test_split(
    x_, y_, train_size=0.6, random_state=42, stratify=y_
)

In [ ]:
print("x_train", x_train.shape)
print("x_val", x_val.shape)
print("x_test", x_test.shape)

Напишите функцию, которая считает расстояние L1 между 2-мя векторами.


In [ ]:
def compute_L1(a, b):
    return np.sum(np.abs(a - b))  # Your code here

Возьмите первую строку из валидационного набора. Посчитайте расстояние L1 от нее до всех строк тренировочного набора.

В простейшем виде напишите `for loop`. Если вы знаете, что вы делаете, можете использовать *векторизацию*.

In [ ]:
# Your code here
a = x_val[0]

l1s = []
for i in range(len(x_train)):
    l1 = compute_L1(a, x_train[i])
    l1s.append(l1)

distances = np.array(l1s)

Найдите индекс минимального расстояния.

Используйте `np.argmin()`.

In [ ]:
indx = distances.argmin()  # Your code here

Выведите первый объект в валидационном наборе и объект, который максимально на него похож в тренировочном (по минимальному расстоянию).

In [ ]:
# Your code here
x_val[0], x_train[indx]

Выведите их метки

In [ ]:
# Your code here
y_val[0], y_train[indx]

Напишите функцию для рассчёта двумерного массива расстояний между двумя выборками (от каждого объекта в первой выборке до каждого объекта во второй выборке).

Рекомендуем заранее создать массив расстояний и заполнить его каким-нибудь очень маленьким числом (например, `np.inf`). Так вы сразу будете отлаживать алгоритм по размерности, а ещё это не будет требовать повторных выделений памяти при росте размера массива.

In [ ]:
def compute_distances(train, sub, distance_func):
    # Your code here
    train_size = len(train)
    sub_size = len(sub)
    distances = np.full((sub_size, train_size), np.inf)
    for i in range(sub_size):
        for j in range(train_size):
            distances[i, j] = distance_func(sub[i], train[j])

    return distances

In [ ]:
distances = compute_distances(x_train, x_val, compute_L1)

In [ ]:
distances.shape

Определите точность Nearest Neighbors классификации на **валидационном** наборе.

Для этого найдите индекс минимального значения для каждой строки (или столбца) массива distances. Количество индексов должно совпадать с количеством объектов в валидационном наборе.

In [ ]:
indx_distances = distances.argmin(axis=1)  # Your code here

In [ ]:
indx_distances.shape

Теперь создадим массив `predicted_class`.

In [ ]:
predicted_class = y_train[indx_distances]

И посмотрим, где класс предсказан правильно, а где нет

In [ ]:
y_val == predicted_class

**Посчитайте точность (accuracy)**

В Python с булевыми значениями можно производить математические операции (`True = 1, False = 0`). Значение accuracy должно быть более $65\%$.

In [ ]:
accuracy = np.mean(y_val == predicted_class)  # Your code here
print(f"Accuracy = {accuracy * 100:.1f}%")

Повторите все этапы классификации, однако в этот раз сделайте **нормализацию** данных перед этим. Величина accuracy должна увеличиться.

In [ ]:
# Your code here
means = x_train.mean(axis=0)
stds = x_train.std(axis=0)
x_train = (x_train - means) / stds
x_val = (x_val - means) / stds

In [ ]:
distances = compute_distances(x_train, x_val, compute_L1)  # Your code here

In [ ]:
min_distances = distances.argmin(axis=1)  # Your code here
predicted_class = y_train[min_distances]  # Your code here
accuracy = np.mean(y_val == predicted_class)  # Your code here
print(f"Accuracy = {accuracy * 100:.1f}%")

**Посчитайте точность (accuracy) для тестового набора**

Теперь учтём, что у нас осталась **тестовая подвыборка**. Проведите необходимые операции и посчитайте accuracy на ней.  

In [ ]:
# Your code here
x_test = (x_test - means) / stds

In [ ]:
distances_test = compute_distances(x_train, x_test, compute_L1)  # Your code here

In [ ]:
distances_test.shape

In [ ]:
y_test.shape

In [ ]:
min_distances_test = distances_test.argmin(axis=1)  # Your code here
predicted_class_test = y_train[min_distances_test]  # Your code here
accuracy_test = np.mean(y_test == predicted_class_test)  # Your code here
print(f"Accuracy = {accuracy_test * 100:.1f}%")

Каков результат? Как вы думаете, почему?

**Дополнительно. Кросс-валидация**

Эта часть задания даёт дополнительные баллы и не обязательна к выполнению.

* Как вы могли отметить, при различных разбиениях на обучение, валидацию и тест вы получаете различные результаты. Попробуйте применить механизм кросс-валидации!

* Вы можете как реализовать этот алгоритм самостоятельно (для проверки сравните индексы объектов в фолдах с разбиением библиотечной реализации), так и использовать готовую библиотечную реализацию. За первый вариант будет больше баллов.

## Формат результата

Получить значения метрик.

## Памятка для преподавателя

Нет сложностей с заданием

# Задание 3. Nearest Neighbors для картинок

В этом задании вы будете применять написанный в задании 2 алгоритм k-NN для работы с картинками.

Импорт необходимых библиотек:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from tqdm.notebook import tqdm
from torchvision import datasets
from sklearn.model_selection import train_test_split

Загрузим датасет с помощью функций torchvision фреймворка PyTorch, с которым мы познакомимся дальше в курсе значительно ближе.

Отметьте, что мы загружаем малую часть датасета для ускорения рассчётов, а также сразу проводим базовую нормировку для изображений. Далее в курсе вы познакомитесь с тем, как эффективнее работать с изображениями.

In [ ]:
dataset = datasets.CIFAR10("content", train=True, download=True)

np.random.seed(42)
data, _, labels, _ = train_test_split(
    dataset.data / 255,  # Normalize
    np.array(dataset.targets),
    train_size=0.1,  # get only fraction of the dataset
    random_state=42,
    stratify=dataset.targets,
)

Посмотрим, что это за датасет.

In [ ]:
data[0][0]

In [ ]:
data.shape

CIFAR-10 — 4-хмерный массив $\small (N, W, H, C)$, где $\small N$ — количество картинок, $\small W$ — ширина картинки, $\small H$ — высота картинки, $\small C$ — количество каналов (RGB).

Создайте subplots с 2-мя строками и 2-мя столбцами и отобразите 4 любых картинки из `data`.
Используйте `plt.imshow()`.

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(4, 4))  # Your code here

ax[0, 0].imshow(data[0])  # Your code here
ax[0, 1].imshow(data[1])  # Your code here
ax[1, 0].imshow(data[2])  # Your code here
ax[1, 1].imshow(data[3])  # Your code here
plt.show()

Разбейте датасет на тренировочный, валидационный и тестовый наборы. Укажите аргументы `random_state=42`, `stratify`.

In [ ]:
# Your code here
x_train, x_, y_train, y_ = train_test_split(
    data, labels, train_size=0.8, random_state=42, stratify=labels
)
# Your code here
x_val, x_test, y_val, y_test = train_test_split(
    x_, y_, train_size=0.5, random_state=42, stratify=y_
)

print("x_train", x_train.shape)
print("x_val", x_val.shape)
print("x_test", x_test.shape)

Возьмите первую картинку из тестового набора и найдите ее ближайшего соседа из тренировочного

In [ ]:
def compute_L1(a, b):
    return np.sum(np.abs(a - b))  # Your code here

In [ ]:
# Your code here
a = x_val[0]

l1s = []
for i in range(len(x_train)):
    l1 = compute_L1(a, x_train[i])
    l1s.append(l1)

distances = np.array(l1s)

In [ ]:
indx = np.argmin(distances)  # Your code here
print(indx)

**Отобразите эти картинки на subplots с `ncols=2`:**

In [ ]:
fig, ax = plt.subplots(ncols=2)  # Your code here
ax[0].imshow(x_test[0])  # Your code here
ax[1].imshow(x_train[indx])  # Your code here
plt.show()

**Посмотрите, какой класс предсказывается:**

In [ ]:
class_pred = y_train[indx]
class_to_idx = dataset.class_to_idx

print(list(class_to_idx.keys())[list(class_to_idx.values()).index(class_pred)])

Возьмите первую картинку из тестового набора и найдите k ее ближайших соседей (k-NN) из тренировочного набора.

Используйте `np.argsort()` или иной способ.

In [ ]:
k = 5
indx = np.argsort(distances)[:k]  # Your code here

Отобразите ближайших соседей в виде subplots:

In [ ]:
fig, ax = plt.subplots(ncols=k, figsize=(16, 4))  # Your code here
# Your code here
ax[0].imshow(x_test[0])
for i in range(1, k):
    ax[i].imshow(x_train[indx[i]])
plt.show()

Посчитайте k-NN для всего датасета.

Чем больше данных, тем дольше процесс. Реализуйте функцию для расчета расстояний. Если вы используете `for loops`, то сделайте к ним *progress bars* с помощью [tqdm](https://github.com/tqdm/tqdm).

Примечание: если используете вложенные циклы, то используйте `tqdm` только на внешнем цикле. Иначе время работы существенно увеличится.

In [ ]:
def compute_distances(sub1, sub2, distance_func):
    # Your code here
    sub1_size = len(sub1)
    sub2_size = len(sub2)
    distances = np.full((sub2_size, sub1_size), np.inf)
    for i in tqdm(range(sub2_size)):
        for j in range(sub1_size):
            distances[i, j] = distance_func(sub2[i], sub1[j])

    return distances

In [ ]:
distances = compute_distances(x_train, x_val, compute_L1)

Теперь найдите k ближайших соседей и предскажите класс. Используйте моду [scipy.stats.mode](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html) по ближайшим найденным соседям.

In [ ]:
def get_accuracy(distances, train_labels, sub_labels, k=5):
    # Your code here
    indexes = np.argsort(distances, axis=1)[:, :k]
    labels_of_top_classes = train_labels[indexes]
    predicted_class, _ = mode(labels_of_top_classes, axis=1, keepdims=True)
    accuracy = np.mean(sub_labels == predicted_class.flatten())
    return accuracy

In [ ]:
accuracy = get_accuracy(distances, y_train, y_val, k)
print(f"Accuracy = {accuracy * 100:.0f}%")

**Посчитайте точность для k=1..100 и постройте график точности от k**

In [ ]:
acc = []
for k in range(1, 100):
    # Your code here
    acc.append(get_accuracy(distances, y_train, y_val, k))

In [ ]:
plt.plot(np.arange(1, 100), acc)  # Your code here
plt.title("Accuracy: validation", fontsize=16)
plt.xlabel("k", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.show()

Поменяйте расстоянние L1 на L2 и сравните точность на всем датасете.

In [ ]:
def compute_L2(a, b):
    return np.sqrt(np.sum((a - b) ** 2))  # Your code here

In [ ]:
distances_l2 = compute_distances(x_train, x_val, compute_L2)

In [ ]:
acc_l2 = []
for k in range(1, 100):
    # Your code here
    acc_l2.append(get_accuracy(distances_l2, y_train, y_val, k=k))

In [ ]:
plt.plot(np.arange(1, 100), acc, label="L1")  # Your code here
plt.plot(np.arange(1, 100), acc_l2, label="L2")  # Your code here
plt.title("Accuracy: validation", fontsize=16)
plt.xlabel("k", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.legend()
plt.show()

In [ ]:
top_acc = max(acc)
top_k = acc.index(top_acc)

print(f"Top acc = {top_acc}, top k = {top_k}")

Теперь, выбрав оптимальные параметры (количество соседей и меру расстояния) с помощью валидационного сета, проверьте качество на **тесте**.

*Примечание 1*. Для минимизации повторения кода можете сделать функцию из кода выше, в которую в качестве аргументов подаются различные наборы данных.

*Примечание 2*. Валидационная выборка позволяет подобрать наилучшие гиперпараметры. Не нужно выводить аналогичные графики зависимости от гиперпарамертров на тесте.

In [ ]:
# Your code here

distances_l1 = compute_distances(x_train, x_test, compute_L1)

In [ ]:
get_accuracy(distances_l1, y_train, y_test, k=top_k)

Совпали ли результаты с валидацией? Как думаете, почему?

**Дополнительно. Замена метрик**

Эта часть задания даёт дополнительные баллы и не обязательна к выполнению.

* Как мы видели в лекции, метрика accuracy довольно редко может быть применима. Попробуйте вывести другую метрику, которая, на ваш взгляд, лучше подходит для данной задачи. Можете использовать библиотечные реализации.

## Вывод

При какой метрике расстояния и с каким k точность будет максимальной?


...

## Формат результата

* График сравнения точности для L1 и L2 при различных k. Выведите на одном графике результаты для валидации и теста.
* Число k, при котором достигается лучшая точность.
* Точность на тесте.

Пример графика:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/Exercises/EX01/result_3_task.png" width="300">

## Памятка для преподавателя

У студентов нет сложностей с этим заданием